In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

Since HoloViews supports a number of rendering backends, the styling options can differ slightly in some cases. However some fundamental concepts such as color cycles, colormapping, setting titles and controlling legends are shared across backends. In this guide we will review how to apply these common styling options to plots.

## Cycles and Palettes

Frequently we want to plot multiple subsets of data, which is made easy by using ``Overlay`` and ``NdOverlay`` objects. When overlaying multiple elements of the same type they will automatically be assigned style cycles or palettes if any are defined.

* ``Cycle``: A Cycle defines a list of discrete styles
* ``Palette``: A Palette defines a continuous color space which will be sampled discretely

### Cycle

A ``Cycle`` can be applied to any of the style options on an element, by default most elements define a ``Cycle`` on the color property, here we will create a set of three ``Points`` object and define a custom ``Cycle``:

In [ ]:
points = (
    hv.Points(np.random.randn(50, 2)      ) *
    hv.Points(np.random.randn(50, 2) + 1  ) *
    hv.Points(np.random.randn(50, 2) * 0.5)
)

cycle = hv.Cycle(['red', 'green', 'blue'])
points.options({'Points': {'color': cycle, 'size': 5}})

#### Defaults

In addition to defining custom color cycles by explicitly defining a list of colors, ``Cycle`` also defines a list of default Cycles generated from bokeh Palettes and matplotlib colormaps:

In [ ]:
hv.Cycle.default_cycles.keys()

To use one of these default Cycles simply construct the Cycle with the corresponding key:

In [ ]:
xs = np.linspace(0, np.pi*2)
curves = hv.Overlay([hv.Curve(np.sin(xs+p)) for p in np.linspace(0, np.pi, 10)])

default_cycle = hv.Cycle('Category20')
curves.options({'Curve': {'color': default_cycle, 'width': 600}})

#### Markers

Cycles are not restricted to defining colors, they may be used to define any style option, e.g. we can use them to cycle over a number of marker styles and sizes, which will be expanded by cycling over each item independently. In this case we are cycling over three Cycles, resulting in the following style combinations:

1. ``{'color': '#30a2da', 'marker': 'x', 'size': 10}``
2. ``{'color': '#fc4f30', 'marker': '^', 'size': 5}``
3. ``{'color': '#e5ae38', 'marker': '+', 'size': 10}``

In [ ]:
color = hv.Cycle(['#30a2da', '#fc4f30', '#e5ae38'])
markers = hv.Cycle(['x', '^', '+'])
sizes = hv.Cycle([10, 5])
points.options({'Points': {'marker': markers, 'size': sizes}})

### Palettes

Palettes work very similarly but instead of treating the colors as a continuous colorspace which will be sampled at regularly spaced intervals. Again they are made automatically available from existing colormaps:

In [ ]:
hv.Palette.colormaps.keys()

As a simple example we will create a Palette from the Spectral colormap and apply it to an Overlay of 6 Ellipses. Comparing it to the spectral ``Cycle``  we can immediately see that the Palette covers the entire color spaces while the Cycle just uses the first 6 colors:

In [ ]:
ellipses = hv.Overlay([hv.Ellipse(0, 0, s) for s in range(6)])

spectral = hv.Palette('Spectral')
spectral_cycle = hv.Cycle('Spectral')

ellipses.relabel('Palette').options({'Ellipse': dict(color=spectral, line_width=5)}) +\
ellipses.relabel('Cycle').options({'Ellipse': dict(color=spectral_cycle, line_width=5)})

## Colormapping

Color cycles and styles are useful for categorical plots and when overlaying multiple subsets, but when we want to map data values to a color it is better to use HoloViews' facilities for color mapping. Certain types will apply colormapping automatically, e.g. on ``Image``, ``QuadMesh`` or ``HeatMap`` types the first value dimension is automatically mapped to the color. In other cases the values to colormap can be declared through the ``color_index``, which may reference any dimension by name or index.


#### Setting a colormap

A HoloViews colormap can take a number of forms, by default HoloViews will make bokeh Palettes and matplotlib colormaps available to reference by name when the respective backend is imported. Additionally it is possible to declare a colormap as a list of hex or HTML colors.

##### Named colormaps

The full list of available named colormaps can be accessed using the ``hv.plotting.list_cmaps``. To provide an overview we will create a layout of all available colormaps (except the reversed versions with the ``'_r'`` suffix).

In [ ]:
%%output backend='matplotlib'
colormaps = hv.plotting.list_cmaps()

spacing = np.linspace(0, 1, 32)[np.newaxis]
opts = dict(aspect=6, xaxis=None, yaxis=None, sublabel_format='')
hv.Layout([hv.Image(spacing, ydensity=1, label=cmap).options(cmap=cmap, **opts)
           for cmap in colormaps if '_r' not in cmap]).options(vspace=0.1, hspace=0.1).cols(8)

To use one of these colormaps simply refer to it by name on the ``cmap`` style option:

In [ ]:
ls = np.linspace(0, 10, 400)
xx, yy = np.meshgrid(ls, ls)
bounds=(-1,-1,1,1)   # Coordinate system: (left, bottom, top, right)
img = hv.Image(np.sin(xx)*np.cos(yy), bounds=bounds)

img.options(cmap='PiYG', colorbar=True, width=400)

##### Custom colormaps

Alternatively custom colormaps can be declared by defining a list of hex colors:

In [ ]:
img.options(cmap=['#0000ff', '#8888ff', '#ffffff', '#ff8888', '#ff0000'], colorbar=True, width=400)

##### Discrete color levels

Lastly existing colormaps can be made discrete by defining an integer number of ``color_levels``:

In [ ]:
img.options(cmap='PiYG', color_levels=10, colorbar=True, width=400)

#### Setting color ranges

Like other dimension ranges the color range can be controlled by setting the range on the dimension. To make this easier we can use the ``.redim.range`` method to set the range on the 'z' value dimension. We will also set some values in the image array to NaN and then set the range to clip the data at 0 and 0.9. By declaring the ``clipping_colors`` option we can control what color values above and below the defined range as well as NaNs are mapped to:

In [ ]:
clipping = {'min': 'red', 'max': 'green', 'NaN': 'gray'}

img_arr = np.sin(xx)*np.cos(yy)
img_arr[:190, :127] = np.NaN
nan_img = hv.Image(img_arr, bounds=bounds)

nan_img.redim.range(z=(0, 0.9)).options(cmap='Blues', colorbar=True, clipping_colors=clipping, width=400)

#### Other options

* ``logz``: Enable logarithmic color scale (e.g. ``logz=True``)
* ``symmetric``: Ensures that the color scale is centered on zero (e.g. ``symmetric=True``)

### Using color_index

As mentioned above when plotting elements which do not automatically map colors to certain dimensions we can use the ``color_index`` option to do so explicitly. This allows colormapping both continuously valued and categorical values.

#### Continuous values

If we map the ``color_index`` to a continuous value we can enable a ``colorbar``:

In [ ]:
polygons = hv.Polygons([{('x', 'y'): hv.Ellipse(0, 0, (i, i)).array(), 'z': i} for i in range(1, 10)[::-1]], vdims='z')

polygons.options(color_index='z', colorbar=True, width=380)

#### Categorical values

Alternatively when mapping to a categorical value we automatically get a legend (which can be disabled using the ``show_legend`` option):

In [ ]:
categorical_points = hv.Points((np.random.rand(100), np.random.rand(100), np.random.choice(list('ABCD'), 100)), vdims='Category')

categorical_points.sort('Category').options(color_index='Category', cmap='Category20', size=5)

#### Explicit color mapping

Instead of using a listed colormap an explicit mapping from category to color is also valid, here we will map the categories 'A', 'B', 'C' and 'D' to specific colors:

In [ ]:
explicit_mapping = {'A': 'blue', 'B': 'red', 'C': 'green', 'D': 'purple'}

categorical_points.sort('Category').options(color_index='Category', cmap=explicit_mapping, size=5)